# Visualization and trajectory analysis of YYY peptide

## Visualization

Before we begin any sort of detailed trajectory analysis, we'll first visualize the trajectory:

In [ ]:
import mdtraj as md
import nglview as nv
import matplotlib.pyplot as plt
import numpy as np
from openmm import app
#%config InlineBackend.figure_format='retina'

In [ ]:
traj = md.load('YYY_sim.dcd', top='YYY.prmtop') # replace YYYYY with your peptide's name
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_representation('cartoon', selection='protein', opacity=0.8)
view.add_representation('licorice', selection='protein'  )
view

## Trajectory Analysis
### End-to-end distance
One thing we want to do is calculate the end-to-end distance of the peptide at each frame (or snapshot) of the trajectory. To do this, we need to identify a specific pair of atoms that can serve as the "ends" of the molecule. Let's choose the C$_\alpha$ atoms of the first residue (1) and last residue (5):

In [ ]:
atoms, bonds = traj.topology.to_dataframe()
atoms[atoms['name'] == 'CA']

Once you have the correct atom indices, insert them into the code below:

In [ ]:
atom_indices = [0,10000] # list of two CA atoms that represent opposite ends of molecule
etoe = md.compute_distances(traj, [atom_indices])
print(etoe.shape)

plt.hist(etoe)
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Counts')
plt.show()

### Solvent-accessible surface area
Another quantity that we'll compute is the solvent-accessible surface area (or polar surface area) of the molecule:

In [ ]:
sasa = md.shrake_rupley(traj)
sasa.shape

You might notice that there are *a lot* of columns in these data. This is because the solvent-accessible surface area calculation outputs this quantity for each atom in the molecule. We'll collapse this down into a single number for the entire molecule below:

In [ ]:
sasa_whole = sasa.sum(axis=1) # this sums up all the columns in each row of sasa
sasa_whole = sasa_whole.reshape((len(sasa), 1))
print(sasa_whole.shape)

plt.hist(sasa_whole)
plt.xlabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

### Hydrogen-bond energy
We can also compute the hydrogen bond energy between each pair of residues. We are using the Kabsch-Sander equation, which is found in mdtraj and you can read more about here: https://mdtraj.org/1.9.3/api/generated/mdtraj.kabsch_sander.html.


In [ ]:
hb = [x.sum() for x in md.kabsch_sander(traj)]
hb = np.array(hb)

hb = hb.reshape((len(hb), 1))
print(hb.shape)

plt.hist(hb)
plt.xlabel('Hydrogen bond energy (kcal/mol)')
plt.ylabel('Counts')
plt.show()

### Dipole moment magnitude

The last thing that we will compute is the magnitude of the overall dipole moment of the molecule. To do this we must: 
1. Determine the charges on each atom of our peptide
2. Determine the positions of each atom at each frame of trajectory
3. Multiply atom charges by the atom positions. 

The following cell will help us get the charges for each atom in our molecule.

In [ ]:
from openmm import NonbondedForce

prmtop = app.AmberPrmtopFile('YYY.prmtop')
system = prmtop.createSystem()

nonbonded = [f for f in system.getForces() if isinstance(f, NonbondedForce)][0]
charges = []
for i in range(system.getNumParticles()):
    charge, _, _ = nonbonded.getParticleParameters(i)
    charges.append(charge.value_in_unit(charge.unit))

In [ ]:
dipole = [np.linalg.norm(x) for x in md.dipole_moments(traj,charges)]
dipole = np.array(dipole)
dipole = dipole.reshape((len(dipole), 1))
dipole.shape

In [ ]:
plt.hist(dipole)
plt.xlabel('Dipole moment magnitude (nm$\cdot$e)')
plt.show()

You might be interested to see a 2D histogram that takes into account two of these quantities, for instance dipole moment and end-to-end distance:

In [ ]:
plt.figure(figsize=(7.5,6))
plt.hist2d(etoe.flatten(), hb.flatten(), bins=(25,25), 
           cmin=1, cmap='plasma')
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Hydrogen bond energy (kcal/mol)')
plt.colorbar()
plt.show()

What about the other combinations of variables, for example solvent-accessible surface area vs. hydrogen-bond energy, or end-to-end distance vs. dipole moment?

Create a 2D histogram for another combination of the four variables below. To do so, change the first two variables in the `plt.hist2d` function, to the desired variable you'd like to visualize. 

In [ ]:
plt.figure(figsize=(7.5,6))
plt.hist2d(dipole.flatten(), sasa_whole.flatten(), bins=(25,25), 
           cmin=1, cmap='plasma')
plt.xlabel('Dipole moment magnitude (nm$\cdot$e)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.colorbar()
plt.show()

Do you see a big difference between this plot and the one above?

## Clustering the trajectory analysis data

One common task is to group all the different structures generated by the simulation into a small number of clusters and then see which characteristics are common to each cluster. In what follows, we'll focus just on clustering the structures from our trajectory using two different algorithms:

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import minmax_scale

X = np.concatenate((etoe, sasa_whole, hb, dipole), axis=1) # construct the input data X
X.shape # check that the shape is correct: (n_samples, n_features)

As discussed in our previous clustering exercise, it's often good to scale the data if the ranges of the features are very different from each other. In this case, the range of the two features are *not* that different, but let's do the min-max scaling nonetheless:

In [ ]:
X2 = minmax_scale(X)
X2

As a first pass, we can try the *k*-means algorithm that we used before. We'll start with 3 clusters, but feel free to experiment with more (or fewer):

In [ ]:
KMclusters = KMeans(n_clusters=3, random_state=1).fit(X2)

plt.figure(figsize=(6,6))
plt.scatter(etoe, sasa_whole, c=KMclusters.labels_, 
            cmap='tab10', alpha=0.5)
plt.xlabel('Dipole moment magnitude (nm$\cdot$e)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

Next, we'll try the Gaussian mixture algorithm... 

The terminology used for the arguments of the `GaussianMixture()` function is somewhat different than those of the `KMeans()` function. One big difference is that `KMeans()` uses `n_clusters`, whereas `GaussianMixture()` uses `n_components`. Also, in order to generate cluster labels using `GaussianMixture()` we need to use `.fit_predict()` rather than `.fit()`.

In [ ]:
GMclusters = GaussianMixture(n_components=4, random_state=0, n_init=50).fit_predict(X2)

plt.figure(figsize=(6,6))
plt.scatter(etoe, sasa_whole, c=GMclusters, 
            cmap='tab10', alpha=0.5)
plt.xlabel('End-to-end distance (nm)')
plt.ylabel('Solvent-accessible surface area (nm$^2$)')
plt.show()

What if we want to plot all of the possible (4 choose 2) combinations together? Well the following code block can help us do that.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(18, 12))

# Plot the data in each subplot
ax[0, 0].scatter(dipole, sasa_whole, c=GMclusters, cmap='tab10', alpha=0.5)
ax[0, 0].set_xlabel('Dipole moment magnitude (nm$\cdot$e)')
ax[0, 0].set_ylabel('Solvent-accessible surface area (nm$^2$)')

ax[0, 1].scatter(dipole, hb, c=GMclusters, cmap='tab10', alpha=0.5)
ax[0, 1].set_xlabel('Dipole moment magnitude (nm$\cdot$e)')
ax[0, 1].set_ylabel('Hydrogen bond energy (kcal/mol)')

ax[1, 0].scatter(dipole, etoe, c=GMclusters, cmap='tab10', alpha=0.5)
ax[1, 0].set_xlabel('Dipole moment magnitude (nm$\cdot$e)')
ax[1, 0].set_ylabel('End-to-end distance (nm)')

ax[1, 1].scatter(hb, sasa_whole, c=GMclusters, cmap='tab10', alpha=0.5)
ax[1, 1].set_xlabel('Hydrogen bond energy (kcal/mol)')
ax[1, 1].set_ylabel('Solvent-accessible surface area (nm$^2$)')

ax[0, 2].scatter(hb, etoe, c=GMclusters, cmap='tab10', alpha=0.5)
ax[0, 2].set_xlabel('Hydrogen bond energy (kcal/mol)')
ax[0, 2].set_ylabel('End-to-end distance (nm)')

ax[1, 2].scatter(etoe, sasa_whole, c=GMclusters, cmap='tab10', alpha=0.5)
ax[1, 2].set_xlabel('End-to-end distance (nm)')
ax[1, 2].set_ylabel('Solvent-accessible surface area (nm$^2$)')

plt.show()

Which graph do you think best separates the data based on the clustering results?

## Your turn #1: evaluating the clustering results
Clearly the k-means algorithm gives different results from the Gaussian mixture algorithm. Based on the 2D histogram above, which clustering algorithm do you think gives more meaningful results?

Use the affinity propagation algorithm to cluster the datapoints. Visualize your results.

1. Did the number of clusters change with this approach? 
2. Peform the affinity propagation algorithm with only two variables of your choosing, how does this change the results?


## Your turn #2 (advanced): linear regression of radius of gyration

The radius of gyration is measure of how spread out the atoms of a peptide are around its center of mass. We can compute this value at each step of our trajectory. Take a look at the documentation for the `compute_rg()` included within `mdtraj`: https://www.mdtraj.org/1.9.8.dev0/api/generated/mdtraj.compute_rg.html a linear regression using the aforementioned variables to fit to the radius of gyration. 

1. Compute the radius of gyration (e.g., `rg = md.compute_rg(...data here...)`).
2. Reshape the radius of gyration list, similar to what was done for the dipole list.
3. Import the `LinearRegression` model and perform a fit linear model of `X` to `rg` target values.
4. Print the regression coefficients and y-intercept. 
5. Use your model to predict your radius of gyration values from your data, `X`. 
6. Visualize the performance of your model by plotting the "measured" vs "computed" radius of gyration.
7. Which of the variables is most important in predicting the radius of gyration?

## Your turn #3: repeat this exercise for a different peptide

Make a copy of this notebook and repeat the MD simulation and analysis for a different peptide. How do data differ for this peptide? How does this affect the clustering and linear regression algorithms? 
